In [ ]:
import geopandas
import pyproj

- https://en.wikipedia.org/wiki/List_of_districts_of_Nepal
- https://mapshaper.org/
- https://geopandas.org/en/stable/docs/user_guide/mapping.html
- https://geopandas.org/en/stable/docs/user_guide/projections.html#re-projecting
- https://proj.org/operations/projections/eqearth.html
- https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html

In [ ]:
DISTRICT_MAP = "raw/nepal/district/npl_admbnda_districts_nd_20201117.shp"

In [ ]:
gdf = geopandas.read_file(DISTRICT_MAP)

In [ ]:
gdf.head()

In [ ]:
gdf.shape

In [ ]:
gdf.crs

In [ ]:
gdf.plot(figsize=(15, 10))

In [ ]:
equal_earth_code = pyproj.CRS("+proj=eqearth").to_epsg()
equal_earth_code

In [ ]:
ee_gdf = gdf.to_crs(epsg=equal_earth_code)

In [ ]:
ee_gdf.crs

In [ ]:
ee_gdf.plot(figsize=(15, 10))

In [ ]:
ee_gdf.head()

---